# Simple Text

In this example we go through a light example of processing a dataset for analyzing text!

The data comes from this [website at Cornell](http://www.cs.cornell.edu/people/pabo/movie-review-data/) and is from Bo Pang and Lillian Lee, A Sentimental Education: Sentiment Analysis Using Subjectivity Summarization Based on Minimum Cuts, Proceedings of ACL 2004.

This contains 1000 positive and 1000 negative reviews.

In [ ]:
# Includes and Standard Magic...
### Standard Magic and startup initializers.

# Load Numpy
import numpy as np
# Load MatPlotLib
import matplotlib
import matplotlib.pyplot as plt
# Load Pandas
import pandas as pd
# Load SQLITE
import sqlite3
# Load Stats
from scipy import stats

# This lets us show plots inline and also save PDF plots if we want them
%matplotlib inline
from matplotlib.backends.backend_pdf import PdfPages
matplotlib.style.use('fivethirtyeight')

# These two things are for Pandas, it widens the notebook and lets us display data easily.
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [ ]:
# These are both in huge directories so let's make 2 data frames.

import glob

all_pos = glob.glob("./data/review_polarity/pos/*")
all_neg = glob.glob("./data/review_polarity/neg/*")
    

In [ ]:
# Let's look at a review and see what's up..
with open('./data/review_polarity/pos/cv839_21467.txt') as f:
    x = f.readlines()
x

In [ ]:
# It's a little messy so let's clean out some of the stuff and join it into one documet.
import re
re.sub(r'[.,\'\"\s\s+]', " ", "".join(x))

In [ ]:
# So now we're read to read and fix up each of the reviews.

revs = []

for fname in all_pos:
    rec = {}
    with open(fname) as f:
        x = f.readlines()
    rec['text'] = re.sub(r'[.,\'\"\s\s+]', " ", "".join(x))
    rec['sentiment'] = 'positive'
    revs.append(rec)

for fname in all_neg:
    rec = {}
    with open(fname) as f:
        x = f.readlines()
    rec['text'] = re.sub(r'[.,\'\"\s\s+]', " ", "".join(x))
    rec['sentiment'] = 'negative'
    revs.append(rec)
    
df_reviews = pd.DataFrame(revs)

In [ ]:
df_reviews

## Now for the fun part...

Now that we have some data to work with let's make some tf-idf vectors

We're going to work with [tf-idf vectorizer from sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)

There are other options and a lot more you could do using sklearn! [See here](https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction).

In [ ]:
# Vectorize the whole thing...
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [ ]:
# Vectorize and play with token sizes...
vec = TfidfVectorizer(min_df = 0.01, 
                      max_df = 0.99, 
                      ngram_range=(2,2)) # play with min_df and max_df

# transform this into a sparse vector!
vec.fit(df_reviews['text'])
tf_idf_sparse = vec.transform(df_reviews["text"])
tf_idf_sparse

What we're doing above is taking the reviews, and computing the tfidf scores for them if we cut off min_df and max_df, so we get letf with fewer words.  We can see the set of words along with the actual tfidf vectors!

In [ ]:
# and for a review we can see the ROW of it's encoding.

print(tf_idf_sparse[0, :])

In [ ]:
# We can now use this to classify the reviews!! but we need to test/train split again.

# Split..
X_train, X_test, y_train, y_test = train_test_split(tf_idf_sparse, 
                                                    df_reviews['sentiment'], 
                                                    test_size=0.2)

In [ ]:
logisticRegr = LogisticRegression(max_iter=100000, class_weight='balanced') 
model = logisticRegr.fit(X_train, y_train)

In [ ]:
# As always we should check our confusing matrix...
# Check and confusion matrix...
from sklearn.metrics import accuracy_score, plot_confusion_matrix
import matplotlib.pyplot as plt
y_pred = model.predict(X_test)
print(accuracy_score(y_test, y_pred))
plot_confusion_matrix(model, X_test, y_test,
                                 display_labels=logisticRegr.classes_,
                                 cmap=plt.cm.Blues, normalize='all')

In [ ]:
logisticRegr.classes_

In [ ]:
logisticRegr.coef_[:,:]

In [ ]:
# We can also do cool stuff like make a dataframe with the words and see which
# have the highest regression weights -- careful here!

# Make a dataframe with the words, coefficients, and classes...
recs = []
for w,i in vec.vocabulary_.items():
    recs.append([str(w)] + list(logisticRegr.coef_[:,i]))
# If we only have one class then we only get weight..
# df_weights = pd.DataFrame(tripples, columns=['word']+list(logisticRegr.classes_))
df_weights = pd.DataFrame(recs, columns=['word', 'weight'])

In [ ]:
df_weights.sort_values('weight', ascending=False)[:25]